In [1]:
from influxdb import InfluxDBClient
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import joblib


In [2]:
influx_client = InfluxDBClient(host='localhost', port=8086, database='iot_s1')

query = 'SELECT * FROM "environment"'
result = influx_client.query(query)

points = list(result.get_points())
df = pd.DataFrame(points)

print(df.head)

<bound method NDFrame.head of                      time  humidity  temperature
0    2024-05-07T04:45:12Z      65.0         25.3
1    2024-05-07T04:45:42Z      64.0         25.8
2    2024-05-07T04:46:12Z      64.0         26.2
3    2024-05-07T04:46:42Z      63.0         26.2
4    2024-05-07T04:47:12Z      63.0         26.2
..                    ...       ...          ...
467  2024-05-07T14:15:32Z      43.0         28.0
468  2024-05-07T14:16:02Z      43.0         28.0
469  2024-05-07T14:16:32Z      43.0         28.0
470  2024-05-07T14:17:02Z      43.0         28.0
471  2024-05-07T14:17:32Z      43.0         28.0

[472 rows x 3 columns]>


In [4]:
df.sort_values('time', inplace=True)

N = 3

features = []
targets = []

for i in range(N, len(df)):
    features.append(df[['temperature', 'humidity']].iloc[i-N:i].values)
    targets.append(df['temperature'].iloc[i])

features = np.array(features)
targets = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

scaler = MinMaxScaler(feature_range=(0, 1))
n_samples, n_timesteps, n_features = X_train.shape

X_train_flattened = X_train.reshape(n_samples * n_timesteps, n_features)
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_train_scaled = X_train_scaled.reshape(n_samples, n_timesteps, n_features)

X_test_flattened = X_test.reshape(X_test.shape[0] * n_timesteps, n_features)
X_test_scaled = scaler.transform(X_test_flattened)
X_test_scaled = X_test_scaled.reshape(X_test.shape[0], n_timesteps, n_features)

joblib.dump(scaler, 'scaler_sq.pkl')

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

model.save('LSTM.h5')

Epoch 1/50


/home/maith/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 772.6896 - val_loss: 758.7107
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 757.9338 - val_loss: 731.4384
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 720.3260 - val_loss: 655.2787
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 619.5807 - val_loss: 483.1529
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 441.9640 - val_loss: 319.1974
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 300.5324 - val_loss: 246.3890
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 241.0527 - val_loss: 210.6346
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 207.0629 - val_loss: 185.1904
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 181.6731 - val_loss: 164.3424
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 163.1416 - val_loss: 146.4154
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 143.5824 - val_loss: 130.6425
Epoch 12/50
12/12 ━━━━━━━━━━━━